# 🏋️‍♀️Fitness identifier Eerste iteratie 

<h2 style="font-size:20px; color:#3B82F6;">Inleiding</h2>

In dit notebook ga ik voor het eerst experimenteren met image classification voor mijn AI-fitnessapp. In de eerste fase van dit project heb ik samen met mijn docent, Bas Michielsen, een geschikte aanpak geformuleerd voor mijn Personal Project.
Bas raadde mij aan om zijn Clothing Classifier-project te gebruiken als eerste testbasis.
Aan de hand van deze test wil ik, samen met hem, verder onderzoeken welke onderdelen ik kan hergebruiken en wat ik eventueel moet aanpassen voor mijn eigen project. In dit notebook documenteer ik mijn eerste test, inclusief opzet, resultaten en reflectie

Voor mijn afbeeldingen ga ik een mix gebruiken uit bestaande datasets en zelfgemaakte afbeeldingen

<h2 style="font-size:20px; color:#3B82F6;">Setup/Imports</h2>

